In [ ]:
import numpy as np
import random

In [ ]:
def MaxMinDiversity(subset: np.ndarray, complete: np.ndarray) -> float:
    """
    Calculate Max-Min Diversity, the minimum distance between all pairs of points in the given subset.
    
    Parameters:
        subset: Subset of points
        complete: Complete graph adjacency matrix containing distances between all pairs of points
    
    Returns:
        min_dist: Max-Min Diversity, the minimum distance between all pairs of points in the subset
    """
    if len(subset) < 2:
        return 0
    
    n = len(subset)
    min_dist = float('inf')
    
    for i in range(n):
        for j in range(i + 1, n):
            dist = complete[subset[i]][subset[j]]
            min_dist = min(min_dist, dist)
    
    return min_dist

In [ ]:
def FairRadius(k: int, complete: np.ndarray) -> np.ndarray:
    """
    Calculate Fair Radius for all points in the dataset.

    Parameters:
        k: Integer parameter k, used to calculate minimum number of points each point needs to cover ⌈n/k⌉
        complete: Complete graph adjacency matrix representing distances between all pairs of points
        
    Returns:
        fair_radius: Fair Radius for all points in the dataset
    """
    n = len(complete)
    min_points = int(np.ceil(n / k))
    fair_radius = []

    for i in range(n):
        distances = complete[i]
        sorted_distances = np.sort(distances)
        point_radius = sorted_distances[min_points-1]
        fair_radius.append(point_radius)
        
    return np.array(fair_radius)


In [ ]:
def CriticalRegion(alpha: float, fair_radius: np.ndarray, complete: np.ndarray) -> tuple[np.ndarray, dict]:
    """
    Generate the Critical Regions by finding a set of centers that cover all points.
    
    Parameters:
        alpha: Fairness parameter
        fair_radius: Array containing fair radius values for all points
        complete: Complete graph adjacency matrix representing distances between all pairs of points
        
    Returns:
        selected_centers: Centers of critical regions
        critical_regions: Dictionary mapping each center to its covered points
    """
    covered_points = set()
    n = len(fair_radius)
    points = set(range(n))
    selected_centers = []
    critical_regions = {}

    while covered_points != points:
        minus = points - covered_points
        c = min(minus, key=lambda x: fair_radius[x])
        selected_centers.append(c)
        for point in minus:
            if complete[point][c] <= 2 * alpha * fair_radius[point]:
                covered_points.add(point)
    selected_centers = np.array(selected_centers)

    for center in selected_centers:
        r_c = fair_radius[center]
        distances = complete[center]
        points_in_circle = np.array([i for i, dist in enumerate(distances) if dist <= alpha * r_c])
        critical_regions[center] = points_in_circle

    return selected_centers, critical_regions


In [ ]:
def IFDM(k: int, alpha: float, complete: np.ndarray, epsilon: float, beta: float, selected_centers: np.ndarray, critical_regions: dict) -> tuple[np.ndarray, tuple[tuple[np.ndarray, int], ...], np.ndarray]:
    """
    Construct an instance of k-clustering under partition matroid constraint corresponding to the given instance of alpha-fair k-clustering.
    
    Parameters:
        k: Target number of clusters
        alpha: Fairness parameter
        complete: Complete graph adjacency matrix with distances
        epsilon: Accuracy parameter (< 1/2)
        beta: Approximation guarantee parameter (≤ 1)
        selected_centers: Centers of critical regions
        critical_regions: Dictionary mapping centers to covered points
        
    Returns:
        P_prime: Augmented point set with duplicated points
        centers_info: Tuple containing center selection info
        d_prime: Modified graph adjacency matrix with distances
    """
    n = len(complete)
    P_0 = np.arange(n)
    
    B_copies = {}
    for center in selected_centers:
        B_copies[center] = critical_regions[center].copy()

    P_prime = list(P_0)
    for center in selected_centers:
        P_prime.extend(B_copies[center])
    P_prime = np.array(P_prime)
    
    k_i = {center: 1 for center in selected_centers}
    
    k_0 = k - len(selected_centers)

    delta = float('inf')
    for i in range(n):
        for j in range(i+1, n):
            if complete[i][j] > 0:
                delta = min(delta, complete[i][j])

    n_prime = len(P_prime)
    d_prime = np.zeros((n_prime, n_prime))

    for i in range(n_prime):
        for j in range(n_prime):
            if i == j:
                d_prime[i][j] = 0
            elif P_prime[i] != P_prime[j]:
                d_prime[i][j] = complete[P_prime[i]][P_prime[j]]
            else:
                d_prime[i][j] = epsilon * beta * delta
                
    return P_prime, ((P_0, k_0), *((B_copies[c], k_i[c]) for c in selected_centers)), d_prime
